# Fractal exploration with Plotly

In [1]:
%%capture
from tqdm import notebook as tqdm
tqdm.tqdm()

import time
import datetime

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import plotly.express as px
import plotly.graph_objects as go

import numpy
import pandas as pd
import math

# Burning Ship and  Mandelbrot Skews

In [43]:
def BuringShip(x,y,power,shift, Yrange,Xrange): 
    zy = y * (Yrange[1] - Yrange[0]) / (imagesize - 1)  + Yrange[0]
    zx = x * (Xrange[1] - Xrange[0]) / (imagesize - 1)  + Xrange[0] 
    z = zx + zy * 1j
    c = z 
    for i in range(max_it):
        if abs(z) > 2.0:
            return exterior(z,i)

        z = (abs(z.real) + abs(z.imag)* 1j) ** power + c +shift
    return interior(z,i)


def MandelbrotSkew(px,py,power,shift, Yrange,Xrange): 
    zy = (py * (Yrange[1] - Yrange[0]) / (imagesize))  + Yrange[0]
    zx = (px * (Xrange[1] - Xrange[0]) / (imagesize))  + Xrange[0]
    z = zx + zy * 1j
    c = z
    for i in range(max_it):
        if abs(z) > 2.0:
            return exterior(z,i)
        z = z ** power + c +shift
    return interior(z,i)

fractals = [BuringShip, MandelbrotSkew]


def interior(z,i): 
    return i
    
def exterior(z,i):
    return i

def iteration_count(z,i):
    return i

def orbit_distance(z,i):
    return abs(z)

def orbit_distance100x(z,i):
    return abs(z)*100

def orbit_distance_log(z,i):
    try:
        return math.log(abs(z))
    except:
         return math.log(abs(z)+1)

def orbit_distance_sinh(z,i):
    return math.sinh(abs(z))
        
maps = [iteration_count, orbit_distance, orbit_distance100x, orbit_distance_log, orbit_distance_sinh ]

In [44]:
def browse_fractals(fractals):
    def fractalpick(fractal):

        global Frac
        Frac = fractal
        return print(fractal.__name__)
    fractal =  interact(fractalpick, fractal=fractals)

# pick fractal    
browse_fractals(fractals)


def powerset(Power):
    global power
    power=Power
    return
    
def shiftset(Shift):
    global shift
    shift=Shift
    return

def exteriorset(Exterior):
    global exterior
    exterior=Exterior

def interiorset(Interior):
    global interior
    interior=Interior
    
    
# pick fractal parameters 
interact(powerset, Power=2.0, description="Power");
interact(shiftset, Shift=0.0, description="shift");

interact(interiorset, Interior=maps);
interact(exteriorset, Exterior=maps);

interactive(children=(Dropdown(description='fractal', options=(<function BuringShip at 0x000001FC920E18C8>, <f…

interactive(children=(FloatSlider(value=2.0, description='Power', max=6.0, min=-2.0), Output()), _dom_classes=…

interactive(children=(FloatSlider(value=0.0, description='Shift', max=1.0), Output()), _dom_classes=('widget-i…

interactive(children=(Dropdown(description='Interior', options=(<function iteration_count at 0x000001FCA07B9C8…

interactive(children=(Dropdown(description='Exterior', options=(<function iteration_count at 0x000001FCA07B9C8…

# Display

In [134]:
# get zoom from last plot or start at ((-1,-2), (1,2)) if figure doesn't exist
if "figzoom" in globals():
    Xrange=figzoom.layout.xaxis.range 
    Yrange=figzoom.layout.yaxis.range 
else:
    Xrange = (-1.0, 1.0)
    Yrange = (-2, 2.0)
    
# set the size of the array/image and interation depth
imagesize = 1080
max_it =256
array = numpy.zeros(shape=(imagesize,imagesize)) 


for i in tqdm.tqdm(range(imagesize)):
    for j in range(imagesize):
        array[i,j] = Frac(i, j, power, shift ,Xrange ,Yrange) 

figzoom = go.FigureWidget(data = 
                go.Heatmap(
                    z=array, 
                    zsmooth='best',
                    showscale=False,
                    colorscale='icefire',
                    x0=Xrange[0], dx=(Xrange[1] - Xrange[0]) / (imagesize),
                    y0=Yrange[0], dy=(Yrange[1] - Yrange[0]) / (imagesize),
                ),
                      layout =
                go.Layout( autosize=False, width=1080, height=1080, 
                    yaxis={'visible': False, 'showticklabels': False},
                    xaxis={'visible': False, 'showticklabels': False},
                    margin=dict(l=0, r=0, b=0,t=0,pad=1))
               )
figzoom


FigureWidget({
    'data': [{'colorscale': [[0.0, '#000000'], [0.0625, '#001f4d'], [0.125,
                   …

In [105]:
#reset zoom
del figzoom

## Export as html

In [44]:
figzoom.write_html("Mandelbrot_orbit_distance.html")

## Save as image

In [99]:
# image naming function
def name_image(fig):

    namestring = Frac.__name__
    namestring += "^" +str(power)
    namestring += "+" +str(shift)

    xmin,xmax =figzoom.layout.xaxis.range
    ymin,ymax =figzoom.layout.yaxis.range
    namestring +="({:.4f},{:.4f}),({:.4f},{:.4f})".format(xmin,xmax,ymin,ymax)
    namestring += exterior.__name__
    namestring += interior.__name__
    
    return namestring

In [100]:
now = str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
figzoom.write_image(name_image(figzoom)+ now +".jpeg")

## Save as high res image

In [135]:
#Hi rez image
Xrange=figzoom.layout.xaxis.range 
Yrange=figzoom.layout.yaxis.range
# set the size of the array/image and interation depth
imagesize = 2160
max_it =256
array = numpy.zeros(shape=(imagesize,imagesize)) 


for i in tqdm.tqdm(range(imagesize)):
    for j in range(imagesize):
        array[i,j] = Frac(i, j, power, shift, Xrange, Yrange)

Hires = go.FigureWidget(data = 
                go.Heatmap(
                    z=array, 
                    zsmooth='best',
                    showscale=False,
                    colorscale='icefire',
                    x0=Xrange[0], dx=(Xrange[1] - Xrange[0]) / (imagesize),
                    y0=Yrange[0], dy=(Yrange[1] - Yrange[0]) / (imagesize),
                ),
                      layout =
                go.Layout( autosize=False, width= 2160, height= 2160, 
                    yaxis={'visible': False, 'showticklabels': False},
                    xaxis={'visible': False, 'showticklabels': False},
                    margin=dict(l=0, r=0, b=0,t=0,pad=1))
               )


now = str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
Hires.write_image(name_image(Hires) + now + ".jpeg")

del Hires